In [ ]:
!pip install refinitiv.data

In [1]:
import pandas as pd
import numpy as np
import os
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing
import logging.config
import datetime
import json

In [2]:
APP_KEY                     = 'aace7028dd2744ab85f73cfceefa2350592ac03b'
RDP_LOGIN                   = '************'
RDP_PASSWORD                = '**********'

# praticamente la chiave da sola non basta. O installi anche Eikon Desktop e fai accesso lì, allora puoi usare solo la APIKEY, altrimenti metti la password (e speriamo che basti!)

# REALTIME_DISTRIBUTION_SERVICE_HOST      = 'THE_HOST:PORT_OF_YOUR_RTDS_HOST'  
# REALTIME_DISTRIBUTION_SERVICE_USER_NAME = 'YOUR_USER_NAME_ON_YOUR_RTDS_HOST' 

## Open the default session

To open the default session ensure you have 
* a '*refinitiv-data.config.json*' in the ***Configuration*** directory 
* **populated the file with your credentials**
* and specified a 'default' session in the config file    

**Note**: To request Historical Pricing data you must be using an RDP Platform (Cloud) session or a Desktop Session(Eikon/Workspace).   
It is not currently possible to request historical data from a deployed Session

Questa pappardella è solo per dirmi che in pratica bisogna installare il programma, fuck!


### Historical Price Events

NOTE: We have not specified any start or end times, so the request defaults to the 20 most recent events

In [3]:
# rd.open_session()
rd.open_session(app_key = APP_KEY)

<refinitiv.data.session.Definition object at 0x20067a76e60 {name='workspace'}>

In [4]:
etf = pd.read_excel("elenco ETF.xlsx")
etf

,Name,ISIN,RIC,Lipper Classification Scheme,Instrument Type,Domicile,Currency,Exchange iNAV
0,Lyxor FTSE MIB Dly (-2X) Inverse Xbear UCITS E...,FR0010446666,XBRMIB.MI,Unclassified,Alternative ETF,France,Euro,NaN
1,Lyxor FTSE MIB Daily (2x) Leveraged UCITS ETF ...,FR0010446658,LEVMIB.MI,Unclassified,Alternative ETF,France,Euro,NaN
2,Lyxor FTSE MIB (DR) UCITS ETF - Dist,FR0010010827,ETFMIB.MI,Equity Italy,Equity ETF,France,Euro,INMIBINAV.PA^F23
3,Lyxor FTSE MIB Daily(-1x) Inverse Bear UCITS E...,FR0010446146,BERMIB.MI,Unclassified,Alternative ETF,France,Euro,NaN
4,UBS LFSBloombergMSCIUS LCrpSustUCI ETF(USD)Ad,LU1215461085,CBSUS.MI,Bond USD Corporates,Bond ETF,Luxembourg,Euro,IUE25INAV.PA^F23
...,...,...,...,...,...,...,...,...
1816,UBS(Irl)ETFplc MSCI EMU ClimParAl UCITS ETF EURAA,IE00BN4Q0L55,EMUPA.MI,Equity EuroZone,Equity ETF,Ireland,Euro,NaN
1817,Global X Video Games & Esports UCITS ETF USD Acc,IE00BLR6Q544,HERUI1.MI,Equity Sector Communication Services,Equity ETF,Ireland,Euro,NaN
1818,Global X Telemedicine & Digital Hlth U ETF USD...,IE00BLR6QB00,EDOCI1.MI,Equity Sector Healthcare,Equity ETF,Ireland,Euro,NaN
1819,UBS(Irl)ETF plc MSCI USA ESG UnvLowCarbSelUEUSDAa,IE00BNC0M350,USESGA.MI,Equity US,Equity ETF,Ireland,Euro,NaN


In [7]:
response = historical_pricing.summaries.Definition(
   "VWCE.MI",
#   ["VWCE.MI", "EQQQ.MI"],
    count=30000,
    interval=historical_pricing.Intervals.DAILY,
    # fields = ["OPEN_PRC","BID","ASK","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
#    fields = "TRDPRC_1",
     fields = ["OPEN_PRC","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
    ).get_data()
# Extract in DataFrame format
response.data.df

VWCE.MI,OPEN_PRC,HIGH_1,LOW_1,TRDPRC_1,TRNOVR_UNS
Date,,,,,
2020-01-14,78.75,78.75,78.75,78.75,78.75
2020-01-15,78.6,78.68,78.5,78.61,15800.78
2020-01-16,78.9,78.9,78.78,78.78,56487.06
2020-01-17,79.57,79.66,79.57,79.66,4059.06
2020-01-20,79.6,79.6,79.6,79.6,1990.0
...,...,...,...,...,...
2023-09-21,102.27,102.33,101.29,101.3,680192.84
2023-09-22,101.1,101.55,101.06,101.46,1188099.08
2023-09-25,101.07,101.4,100.6,101.4,4730890.08


In [15]:
errori = []
errori_nomi = []
for nome in list(etf.RIC):
    try:
        response = historical_pricing.summaries.Definition(
            #   "VWCE.MI",
            nome,
            count=30000,
            interval=historical_pricing.Intervals.DAILY,
            # fields = ["OPEN_PRC","BID","ASK","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
            fields = ["OPEN_PRC","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
        ).get_data()
        response.data.df.to_csv("dati\\" + nome + ".csv",sep=";")
    except rd.errors.RDError as e:
        print("Error code :", e.code," Message :", e.message)
        errori.append((e.code,e.message))
        errori_nomi.append(nome)

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

...

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16px;
}
</style>
</head>
<!-- Request 
Referer: https://emea1-apps)
Error code : 1  Message : No data to return, please check errors: ERROR: No successful response.
(403, <html>
<head>
<title>Forbidden</title>
<style type="text/css">
body {
  background-color: #242424;
  color: #c8c7c7;
  font-family: Calibri, Helvetica, Arial, sans-serif;
  font-size: 16p

In [16]:
print(errori_nomi)

['XBRMIB.MI', 'LEVMIB.MI', 'ETFMIB.MI', 'BERMIB.MI', 'CBSUS.MI', 'HYS.MI', 'JT13.MI', 'GGOVI.MI', 'JREB.MI', 'EGE.MI', 'ITAMID.MI', 'ICFP.MI', 'SBEME.MI', 'IEBB.MI', 'FEDF.MI', 'WLDX.MI', 'JR15.MI', 'EMUL.MI', 'HDRO.MI', 'SPMVV.MI', 'PPEU.MI', 'PSRE.MI', 'DRGN.MI', 'PSRF.MI', 'JRUB.MI', 'GHYEH.MI', 'LVLE.MI', 'SPHC.MI', 'ITAPIR.MI', 'CATHP.MI', 'PSDE.MI', 'KSTRA.MI', 'VNTM.MI', 'VDTM.MI', '3ITS.MI', '3ITL.MI', 'U10H.MI', '3BAL.MI', 'VIXL.MI', '3CFL.MI', 'EUS3.MI', '3BRS.MI', 'USE3.MI', 'U13H.MI', 'GOL1L.MI', '3WHL.MI', 'INFHL.MI', 'IITA.MI', 'GAS3L.MI', 'CBSUSA.MI', 'GAS3S.MI', 'EUES.MI', 'MIB3S.MI', 'MIB3L.MI', '3BTL.MI', 'DXJF.MI', '3SUL.MI', 'COBO.MI', 'WLDC.MI', 'GBSE.MI', 'USRIH.MI', 'EUJP.MI', 'WSPX.MI', '5BTS.MI', '3TYL.MI', 'USEU.MI', '3EML.MI', 'EUS5.MI', '3SCR.MI', 'USE5.MI', 'TINE.MI', 'SAUA.MI', 'ECECOP.MI', 'ECECRD.MI', 'OATS5.MI', 'USYH.MI', 'NDQ3S.MI', 'ITBL.MI', '3LCR.MI', 'BTPS1.MI', 'UNCA.MI', 'URAM.MI', 'MIBA.MI', 'CEUDIM.MI', 'EBUL.MI', 'ITPD.MI', 'GLDVDE.MI', '3LSP

In [17]:
pd.Series(errori_nomi).to_csv("etf_errati.csv")

In [ ]:
rd.close_session()

# Pezzi che per ora non servono

In [ ]:
response = historical_pricing.events.Definition("VOD.L").get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
# JSON format (display only 200 characters of headers + data to minimise output)
print(json.dumps(response.data.raw['universe'], indent=2),
      'Headers:',
      json.dumps(response.data.raw['headers'], indent=2)[0:200],
      'Data:',
      json.dumps(response.data.raw['data'], indent=2)[0:200])

In [ ]:
response = historical_pricing.events.Definition(
    universe = "AAPL.O", 
    start = datetime.timedelta(-1), 
    end = datetime.timedelta(0),
    adjustments = [
        historical_pricing.Adjustments.EXCHANGE_CORRECTION,
        historical_pricing.Adjustments.MANUAL_CORRECTION
    ]).get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
response = historical_pricing.summaries.Definition(
    universe = "IBM.N", 
    interval = historical_pricing.Intervals.ONE_MINUTE,     # Supported intervals: ONE_MINUTE, FIVE_MINUTES, TEN_MINUTES, THIRTY_MINUTES, ONE_HOUR
    count = 500,
    sessions = [
        historical_pricing.MarketSession.PRE, 
        historical_pricing.MarketSession.NORMAL, 
        historical_pricing.MarketSession.POST
    ]).get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
response = historical_pricing.summaries.Definition("IBM.N").get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
response = historical_pricing.summaries.Definition(
    universe = "VOD.L",
    interval = historical_pricing.Intervals.DAILY,          # Supported intervals: DAILY, WEEKLY, MONTHLY, QUARTERLY, YEARLY.
    count = 10,
    fields = ["BID", "ASK", "OPEN_PRC", "HIGH_1", "LOW_1", "TRDPRC_1", "NUM_MOVES", "TRNOVR_UNS"]
).get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
try:
    response = historical_pricing.events.Definition("BADRIC").get_data()
    print(response.data.df)
except rd.errors.RDError as e:
    print("Error code :", e.code)
    print("Error message:", e.message)

# Tentativo di farlo funzionare senza installare nulla

In [ ]:
# session = rd.session.desktop.Definition(app_key = APP_KEY).get_session()
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
).get_session()

In [ ]:
# rd.open_session()
# rd.close_session()

In [ ]:
session.open()

In [ ]:
session.set_log_level(logging.DEBUG)
session.open()
# To set logging back to default value
session.set_log_level(logging.WARNING)

In [ ]:
# Callback to handle session state changes
def on_state_cb(state_code, state_msg, session):
    print(f"Session State:{state_code} | {state_msg}")
# Callback to handle session events
def on_event_cb(event_code, event_msg, session):
    print(f"Session event:{event_code} | {event_msg}")

# Set your callbacks for handing the session events
session.on_event(on_event_cb)
session.on_state(on_state_cb)

In [ ]:
session.info

In [ ]:
session.close()